# 数据读取

In [1]:
import pandas as pd
import random
import math
from collections import defaultdict

# 读取用户数据集，指定列名
users = pd.read_csv('users.dat', sep='::', engine='python', header=None,
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
print("Users 数据集预览：")
print(users.head())

# 读取电影数据集，指定列名
movies = pd.read_csv('movies.dat', sep='::', engine='python', header=None, encoding='latin1',
                     names=['MovieID', 'Title', 'Genres'])
    
# 将 Genres 列中的字符串按 "|" 分割，转换成列表
movies['Genres'] = movies['Genres'].apply(lambda x: x.split('|'))

print("\nMovies 数据集预览：")
print(movies.head())



# 读取 ratings 数据集
ratings = pd.read_csv('ratings.dat', sep='::', engine='python', header=None,
                        names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
print("Ratings 数据集预览：")
print(ratings.head())

# 对于隐反馈数据，只需要记录 (UserID, MovieID) 对，忽略 Rating 和 Timestamp
data = list(zip(ratings['UserID'], ratings['MovieID']))


Users 数据集预览：
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455

Movies 数据集预览：
   MovieID                               Title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                             Genres  
0   [Animation, Children's, Comedy]  
1  [Adventure, Children's, Fantasy]  
2                 [Comedy, Romance]  
3                   [Comedy, Drama]  
4                          [Comedy]  
Ratings 数据集预览：
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408

# 数据集划分

In [2]:
import random

def SplitData(data, M, k, seed):
    """
    将数据集 data 均匀随机划分为 M 份，
    k 为当前划分的编号（0 <= k < M），
    如果某条数据被随机选中为 k，则划分到测试集，
    否则划分到训练集。

    参数:
    - data: 用户行为数据，格式为 [(user, item), ...]
    - M: 划分份数，例如 8
    - k: 本次实验选用的测试集编号，0 <= k < M
    - seed: 随机种子，用于保证实验可重复

    返回:
    - train: 训练集列表
    - test: 测试集列表
    """
    test = []
    train = []
    random.seed(seed)
    for user, item in data:
        # 随机生成 0 到 M-1 之间的一个整数
        if random.randint(0, M - 1) == k:
            test.append([user, item])
        else:
            train.append([user, item])
    return train, test


# 这里选择 M=8，k=0 作为一次实验（可循环不同 k 得到多个结果）
M = 8
k = 0
seed = 42
train_data, test_data = SplitData(data, M, k, seed)

print(f"\n训练集大小: {len(train_data)}, 测试集大小: {len(test_data)}")



训练集大小: 874948, 测试集大小: 125261


# 用户-物品集 & 用户相似度

## 用户物品集

In [3]:
# ---------------------------
# 构建用户-物品字典（训练集）
# ---------------------------
def build_user_item_dict(train_data):
    """
    根据训练集，生成 {user: set(items)} 结构
    """
    user_item = defaultdict(set)
    for user, item in train_data:
        user_item[user].add(item)
    return user_item

user_item = build_user_item_dict(train_data)


## 物品相似度

### 普通版本（可选IUF）

In [ ]:
# ---------------------------
# 计算item相似度
# ---------------------------
def calc_item_similarity(user_item):
    """
    计算物品之间的相似度
    参数:
      user_item: dict, 格式为 {user: set(items)} 表示每个用户交互过的物品集合
    返回:
      item_similarity: dict, 格式为 {i: {j: similarity}}，表示物品 i 与物品 j 的相似度
    """
    # 构建物品-用户倒排表
    item_users = defaultdict(set)
    for user, items in user_item.items():
        for i in items:
            item_users[i].add(user)
    
    # 共现矩阵，用于统计同时出现在同一用户行为中的物品对次数
    co_occurrence = defaultdict(lambda: defaultdict(int))
    # 统计每个物品被交互的次数（即用户数）
    item_count = {i: len(users) for i, users in item_users.items()}
    
    # 遍历每个用户的交互物品，更新共现矩阵
    for user, items in user_item.items():
        items = list(items)
        for i in range(len(items)):
            for j in range(i + 1, len(items)):
                item_i = items[i]
                item_j = items[j]
                co_occurrence[item_i][item_j] += 1
                co_occurrence[item_j][item_i] += 1  # 对称关系

                #对于ItemCF-IUF,只需要 += 1 / math.log(1+len(items))

    # 根据共现矩阵计算余弦相似度
    item_similarity = defaultdict(dict)
    for i, related_items in co_occurrence.items():
        for j, co_count in related_items.items():
            # 注意：如果某个物品没有交互记录，则 item_count[i] 为 0，通常不会出现这种情况
            sim = co_count / math.sqrt(item_count[i] * item_count[j])
            item_similarity[i][j] = sim

    return item_similarity

item_similarity = calc_item_similarity(user_item)

### 归一化版本

In [24]:
# ---------------------------
# 带归一化的
# ---------------------------
def calc_item_similarity_norm(user_item):
    """
    计算物品之间的相似度
    参数:
      user_item: dict, 格式为 {user: set(items)} 表示每个用户交互过的物品集合
    返回:
      item_similarity: dict, 格式为 {i: {j: similarity}}，表示物品 i 与物品 j 的相似度
    """
    # 构建物品-用户倒排表
    item_users = defaultdict(set)
    for user, items in user_item.items():
        for i in items:
            item_users[i].add(user)
    
    # 共现矩阵，用于统计同时出现在同一用户行为中的物品对次数
    co_occurrence = defaultdict(lambda: defaultdict(int))
    # 统计每个物品被交互的次数（即用户数）
    item_count = {i: len(users) for i, users in item_users.items()}
    
    # 遍历每个用户的交互物品，更新共现矩阵
    for user, items in user_item.items():
        items = list(items)
        for i in range(len(items)):
            for j in range(i + 1, len(items)):
                item_i = items[i]
                item_j = items[j]
                co_occurrence[item_i][item_j] += 1
                co_occurrence[item_j][item_i] += 1  # 对称关系

                #对于ItemCF-IUF,只需要 += 1 / math.log(1+len(items))

    # 根据共现矩阵计算余弦相似度
    item_similarity = defaultdict(dict)
    for i, related_items in co_occurrence.items():
        for j, co_count in related_items.items():
            # 注意：如果某个物品没有交互记录，则 item_count[i] 为 0，通常不会出现这种情况
            sim = co_count / math.sqrt(item_count[i] * item_count[j])
            item_similarity[i][j] = sim

    # ---------------------------
    # 归一化处理：对每个物品 i，将与其它物品的相似度除以其最大相似度
    # 参考: George Karypis 的 "Evaluation of Item-based Top-N Recommendation Algorithms"
    # ---------------------------
    for i, sim_dict in item_similarity.items():
        max_sim = max(sim_dict.values(), default=0)
        if max_sim > 0:
            for j in sim_dict:
                sim_dict[j] /= max_sim

    return item_similarity

item_similarity_norm = calc_item_similarity_norm(user_item)

In [11]:
# 为方便查找，构造一个从 MovieID 到 Title 的字典
movie_titles = pd.Series(movies.Title.values,index=movies.MovieID).to_dict()

# 遍历物品相似度字典，打印出每个电影及其相似电影信息
print("物品相似度情况（仅显示前5个电影的相似度信息）：")
for idx, (movie_id, sim_dict) in enumerate(item_similarity.items()):
    if idx >= 5:  # 只显示前5个电影
        break
    title = movie_titles.get(movie_id, "Unknown")
    print(f"电影 {movie_id} - {title}:")
    # 遍历与该电影相似的其他电影
    for other_id, sim in sim_dict.items():
        other_title = movie_titles.get(other_id, "Unknown")
        print(f"    与电影 {other_id} - {other_title} 的相似度: {sim:.4f}")
    print()

物品相似度情况（仅显示前5个电影的相似度信息）：
电影 1 - Toy Story (1995):
    与电影 2692 - Run Lola Run (Lola rennt) (1998) 的相似度: 0.3015
    与电影 1028 - Mary Poppins (1964) 的相似度: 0.3951
    与电影 1029 - Dumbo (1941) 的相似度: 0.3416
    与电影 1287 - Ben-Hur (1959) 的相似度: 0.2747
    与电影 1545 - Ponette (1996) 的相似度: 0.0793
    与电影 527 - Schindler's List (1993) 的相似度: 0.4418
    与电影 914 - My Fair Lady (1964) 的相似度: 0.2764
    与电影 531 - Secret Garden, The (1993) 的相似度: 0.2290
    与电影 150 - Apollo 13 (1995) 的相似度: 0.4070
    与电影 1566 - Hercules (1997) 的相似度: 0.3369
    与电影 3105 - Awakenings (1990) 的相似度: 0.3002
    与电影 2340 - Meet Joe Black (1998) 的相似度: 0.2098
    与电影 1193 - One Flew Over the Cuckoo's Nest (1975) 的相似度: 0.3415
    与电影 938 - Gigi (1958) 的相似度: 0.1467
    与电影 1961 - Rain Man (1988) 的相似度: 0.3687
    与电影 1836 - Last Days of Disco, The (1998) 的相似度: 0.1369
    与电影 1197 - Princess Bride, The (1987) 的相似度: 0.4880
    与电影 1962 - Driving Miss Daisy (1989) 的相似度: 0.2947
    与电影 3114 - Toy Story 2 (1999) 的相似度: 0.5471
    与电影 48 - P

# 推荐ItemCF

In [5]:
# ---------------------------
# 推荐函数（ItemCF）
# ---------------------------
def ItemCF(user, user_item, item_similarity, N=10):
    """
    基于物品相似度为目标用户推荐物品
    参数:
      user: 当前目标用户
      user_item: dict, 格式为 {user: set(items)}，用户的行为数据
      item_similarity: dict, 格式为 {i: {j: similarity}}，物品相似度矩阵
      N: 推荐列表的长度
    返回:
      rec_list: 推荐的物品列表（按照得分降序排序）
    """
    # 获取当前用户已交互的物品集合
    interacted_items = user_item.get(user, set())
    scores = defaultdict(float)
    
    # 对于用户已经交互的每个物品
    for i in interacted_items:
        # 遍历与该物品相似的其他物品及相似度
        for j, sim in item_similarity.get(i, {}).items():
            if j in interacted_items:
                continue  # 已交互的物品不再推荐
            scores[j] += sim

    # 根据累加得分排序，取前 N 个物品
    ranked_items = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:N]
    rec_list = [item for item, score in ranked_items]
    return rec_list

# 评测函数

In [9]:
# ---------------------------
# 评测函数
# ---------------------------
def evaluate(user_item, test_data, item_similarity, N=10):
    """
    在测试集上评测推荐效果，计算 Precision、Recall、Coverage 和 Popularity
    """
    # 构建测试集的用户-物品字典
    test_user_item = defaultdict(set)
    for user, item in test_data:
        test_user_item[user].add(item)
    
    # 计算所有物品的流行度（出现次数），用于流行度评估
    item_popularity = defaultdict(int)
    for items in user_item.values():
        for item in items:
            item_popularity[item] += 1
    all_items = set(item_popularity.keys())
    
    hit = 0
    total_precision = 0
    total_recall = 0
    recommended_items_set = set()
    total_test_items = 0

    # 用于计算推荐列表的平均流行度
    total_popularity = 0
    total_rec = 0
    
    users = set(list(user_item.keys()) + list(test_user_item.keys()))
    
    for user in users:
        rec_items = ItemCF(user, user_item, item_similarity, N=N)
        true_items = test_user_item.get(user, set()) #从测试集中获取当前用户的真实交互物品集合
        hit += len(set(rec_items) & true_items) #计算当前用户推荐列表中有多少物品出现在真实交互集合中，即交集的大小
        
        total_precision += N #precision 分母
        total_recall += len(true_items) #recall 分母
        
        recommended_items_set.update(rec_items)
        
        total_test_items += len(true_items) #统计所有用户在测试集中真实交互物品的总数量，计算召回率
        
        #计算流行度
        for item in rec_items:
            total_popularity += math.log(1 + item_popularity.get(item, 0))  #取对数平滑数据防止极值
            total_rec += 1

    precision = hit / (total_precision + 1e-10)
    recall = hit / (total_recall + 1e-10)
    coverage = len(recommended_items_set) / len(all_items)
    popularity = total_popularity / (total_rec + 1e-10)
    
    return precision, recall, coverage, popularity

# 实验运行和结果输出

### 普通

In [12]:
# ---------------------------
# 执行一次实验并输出评测结果
# ---------------------------

N = 10   # 推荐列表长度
precision, recall, coverage, popularity = evaluate(user_item, test_data, item_similarity,N=N)
print(f"\n评测结果：")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"Coverage:  {coverage:.4f}")
print(f"Popularity:{popularity:.4f}")


评测结果：
Precision: 0.2085
Recall:    0.1005
Coverage:  0.1267
Popularity:7.3978


### 归一化

In [25]:
# ---------------------------
# 执行一次实验并输出评测结果
# ---------------------------

N = 10   # 推荐列表长度
precision, recall, coverage, popularity = evaluate(user_item, test_data, item_similarity_norm,N=N)
print(f"\n评测结果：")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"Coverage:  {coverage:.4f}")
print(f"Popularity:{popularity:.4f}")


评测结果：
Precision: 0.2122
Recall:    0.1023
Coverage:  0.1440
Popularity:7.3513
